# Setup

In [1]:
from optibook.synchronous_client import Exchange

import time
import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

kj/filesystem-disk-unix.c++:1734: warning: PWD environment variable doesn't match current directory; pwd = /


Setup was successful.


# Connecting to the Exchange

In [2]:
exchange = Exchange()
_ = exchange.connect()

2025-02-01 14:15:32,029 [asyncio   ] [MainThread  ] Using selector: EpollSelector


# Getting market information

In [3]:
# Load all instruments (products) available on the exchange
instruments = exchange.get_instruments()
instruments

{'SEMIS_ETF_EU': Instrument(instrument_id=SEMIS_ETF_EU, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.INDEX_TRACKING_ETF, index_id=SEMIS, instrument_group=SEMIS, paused=False, expired=False),
 'SEMIS_ETF_US': Instrument(instrument_id=SEMIS_ETF_US, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.INDEX_TRACKING_ETF, index_id=SEMIS, instrument_group=SEMIS, paused=False, expired=False),
 'ASML': Instrument(instrument_id=ASML, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.STOCK, instrument_group=ASML, paused=False, expired=False),
 'AMD': Instrument(instrument_id=AMD, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.STOCK, instrument_group=AMD, paused=False, exp

In [4]:
# Define a variable storing the instrument_id of the product we are interested in
instrument_id = 'DEMO'

### Order Book

In [5]:
# Load current limit order book for the instrument
exchange.get_last_price_book(instrument_id)

2025-02-01 14:27:01,920 [client    ] [Thread-4 (_thread_entry_point)] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


### Public tradeticks

In [12]:
# Load and store a list of all public tradeticks since the instantiation of the Exchange (upto a max limit)
# These are trades between any participant, you may or may not have participated in these
trade_tick_history = exchange.get_trade_tick_history(instrument_id)

# Display last 5
trade_tick_history[-5:]

[]

In [13]:
# Poll all public tradeticks since the last time this method was called
exchange.poll_new_trade_ticks(instrument_id)

[]

### Private trades

In [14]:
# Load and store a list of all private trades you participated in since the instantiation of the Exchange (upto a max limit)
trade_history = exchange.get_trade_history(instrument_id)

# Display last 5
trade_history[-5:]

[]

In [15]:
# Poll all private trades since the last time this method was called
exchange.poll_new_trades(instrument_id)

[]

### Position & PnL

In [16]:
# Load current positions in all instruments
exchange.get_positions()

{'NVDA': 0, 'AMD': 0, 'ASML': 0, 'SEMIS_ETF_US': 10, 'SEMIS_ETF_EU': 0}

In [17]:
# Load current positions in all instruments, including cash component (how much did we invest or gain by trading each instrument)
exchange.get_positions_and_cash()

{'NVDA': {'volume': 0, 'cash': 0.0},
 'AMD': {'volume': 0, 'cash': 0.0},
 'ASML': {'volume': 0, 'cash': 0.0},
 'SEMIS_ETF_US': {'volume': 10, 'cash': -1177.6999999999996},
 'SEMIS_ETF_EU': {'volume': 0, 'cash': 0.0}}

In [18]:
# Current PnL: sum of all cash components and value of the positions (evaluated against last-traded price) 
exchange.get_pnl()

-40.69999999999959

2025-02-01 14:13:25,742 [client    ] [Thread-4 (_thread_entry_point)] Forcing a disconnect due to an error: Disconnecting because heart beat was not responded to in time.


# Orders

### Inserting

In [ ]:
# Insert bid LIMIT order - this trades against any current orders, and any remainders become new resting orders in the book
# The returned value is a 'InsertOrderResponse' which, if successfull, contains the order_id of the order
exchange.insert_order(instrument_id, price=10, volume=5, side='bid', order_type='limit')

In [ ]:
# Insert ask LIMIT order - this trades against any current orders, and any remainders become new resting orders in the book
exchange.insert_order(instrument_id, price=20, volume=5, side='ask', order_type='limit')

In [ ]:
# Insert bid IOC order - this order trades against any resting volume which has an equal or better price, but does not remain in the 
# book if any volume is unfilled
exchange.insert_order(instrument_id, price=10, volume=5, side='bid', order_type='ioc')

In [ ]:
# Insert ask IOC order - this order trades against any resting volume which has an equal or better price, but does not remain in the 
# book if any volume is unfilled
exchange.insert_order(instrument_id, price=20, volume=5, side='ask', order_type='ioc')

### Deleting & amending

In [ ]:
# Load a list of own outstanding/resting orders
exchange.get_outstanding_orders(instrument_id)

In [ ]:
# Delete an order by order_id, the returned DeleteOrderRequest contains a boolean showing whether the delete was succesful
exchange.delete_order(instrument_id, order_id=1234)

In [ ]:
# Delete all outstanding orders for an instrument
exchange.delete_orders(instrument_id)

In [ ]:
# Change (amend) the volume of an existing resting order by order_id, the returned AmendOrderResponse contains a boolean showing whether the amend was succesful
exchange.amend_order(instrument_id, order_id=1234, volume=30)

# Code Snippets

Combining a few of the exchange interactions above, we can write a code snippet to aggressively trade out of all currently held positions using IOC orders. 

That means selling all products you are long, and buying back all you are short. 

This is useful for starting with a clean slate, but doing so will come at a potentially large cost. 

You are not guaranteed to end up with a zero position afterwards, only that volume which is also available to trade in the market will be bought/sold.

In [ ]:
MIN_SELLING_PRICE = 0.10
MAX_BUYING_PRICE = 100000.00

positions = exchange.get_positions()
pnl = exchange.get_pnl()

print(f'Positions before: {positions}')
print(f'\nPnL before: {pnl:.2f}')

print(f'\nTrading out of positions')
for iid, pos in positions.items():
    if pos > 0:
        print(f'-- Inserting sell order for {pos} lots of {iid}, with limit price {MIN_SELLING_PRICE:.2f}')
        exchange.insert_order(iid, price=MIN_SELLING_PRICE, volume=pos, side='ask', order_type='ioc')
    elif pos < 0:
        print(f'-- Inserting buy order for {abs(pos)} lots of {iid}, with limit price {MAX_BUYING_PRICE:.2f}')
        exchange.insert_order(iid, price=MAX_BUYING_PRICE, volume=-pos, side='bid', order_type='ioc')
    else:
        print(f'-- No initial position in {iid}, skipping..')
    
    time.sleep(0.10)

time.sleep(1.0)

positions = exchange.get_positions()
pnl = exchange.get_pnl()
print(f'\nPositions after: {positions}')
print(f'\nPnL after: {pnl:.2f}')